In [1]:
PROJECT = "area51-lab"
DATASET = "raw"
TABLE = "testing"
TABLE_PATH = f"projects/{PROJECT}/datasets/{DATASET}/tables/{TABLE}"
WRITE_STREAM = f"{TABLE_PATH}/streams/_default"

%env GOOGLE_APPLICATION_CREDENTIALS=/Users/klim/Desktop/area51.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/klim/Desktop/area51.json


In [2]:
import sys

sys.path.insert(0, "../")
sys.path

['../',
 '/Users/klim/Projects/twitter-trends-crawling/src/notebooks',
 '/Users/klim/.pyenv/versions/3.9.13/lib/python39.zip',
 '/Users/klim/.pyenv/versions/3.9.13/lib/python3.9',
 '/Users/klim/.pyenv/versions/3.9.13/lib/python3.9/lib-dynload',
 '',
 '/Users/klim/Library/Caches/pypoetry/virtualenvs/twitter-trends-crawling-8JX5Pgav-py3.9/lib/python3.9/site-packages']

In [7]:
from datetime import datetime as dt, timezone as tz
from google.cloud.bigquery_storage import (
    BigQueryWriteClient,
    AppendRowsRequest, 
    WriteStream,
    ProtoRows,
    ProtoSchema
)
from google.protobuf import descriptor_pb2
import model.bqmodel_pb2 as model

client = BigQueryWriteClient()
parent = client.table_path(PROJECT, DATASET, TABLE)
write_stream = client.create_write_stream(
    parent=parent, 
    write_stream=WriteStream(type_=WriteStream.Type.COMMITTED)
)

write_stream.name

'projects/area51-lab/datasets/raw/tables/testing/streams/Cic3MzkxZDQ2MC0wMDAwLTI3MWYtYjkzNS1kNGY1NDdmMzBmYjA6czA'

In [10]:
proto_descriptor = descriptor_pb2.DescriptorProto()
model.User.DESCRIPTOR.CopyToProto(proto_descriptor)

delta = dt.now() - dt(1970, 1, 1)
ts_value = int(delta.total_seconds()) * 1000000 + int(delta.microseconds)

row = model.User()
row.id = "123123132131"
row.username = "123123132131"
row.display_name = "123123132131"
row.created_timestamp = ts_value
row.description = "123123132131"
row.verified = True
row.location = "123"
row.followers_count = 10
row.friends_count = 10
row.statuses_count = 10
row.favourites_count = 10
row.data_ts = ts_value

row

id: "123123132131"
username: "123123132131"
display_name: "123123132131"
description: "123123132131"
created_timestamp: 1659307933825412
verified: true
location: "123"
followers_count: 10
friends_count: 10
statuses_count: 10
favourites_count: 10
data_ts: 1659307933825412

In [11]:
proto_schema = ProtoSchema(proto_descriptor=proto_descriptor)
proto_rows = ProtoRows(serialized_rows=[
    row.SerializeToString(),
    row.SerializeToString()
])
proto_data = AppendRowsRequest.ProtoData(
    writer_schema=proto_schema,
    rows=proto_rows
)

request = AppendRowsRequest(write_stream=write_stream.name, proto_rows=proto_data)

resp = client.append_rows(requests=iter([request]))
resps = list(resp)

print(resps)

for resp in resps:
    print(resp.append_result)
    if resp.append_result:
        pass

InvalidArgument: 400 Invalid proto schema: BqMessage.proto: User.label: The [proto3_optional=true] option may only be set on proto3fields, not User.label Entity: projects/area51-lab/datasets/raw/tables/testing/streams/Cic3MzkxZDQ2MC0wMDAwLTI3MWYtYjkzNS1kNGY1NDdmMzBmYjA6czA

In [3]:
proto_descriptor = descriptor_pb2.DescriptorProto()
model.Tweet.DESCRIPTOR.CopyToProto(proto_descriptor)

delta = dt.now() - dt(1970, 1, 1)
ts_value = int(delta.total_seconds()) * 1000000 + int(delta.microseconds)

row = model.Tweet()
row.id = "123123132131"
row.url = "123123132131"
row.content = "123123132131"
row.created_timestamp = ts_value
row.user_id = "123123132131"
row.retweets_count = 1
row.quote_tweets_count = 1
row.likes_count = 1
# row.hashtags.exten
row.replies.extend(["123123132131", "123123132131"])
row.source_label = "123123132131"
row.

row

AttributeError: Assignment not allowed to repeated field "hashtags" in protocol message object.

In [6]:
proto_schema = ProtoSchema(proto_descriptor=proto_descriptor)
proto_rows = ProtoRows(serialized_rows=[
    row.SerializeToString(),
    row.SerializeToString(),
    row.SerializeToString()
])
proto_data = AppendRowsRequest.ProtoData(
    writer_schema=proto_schema,
    rows=proto_rows
)

request = AppendRowsRequest(write_stream=write_stream.name, proto_rows=proto_data)

resp = client.append_rows(requests=iter([request]))
resps = list(resp)

print(resps)

for resp in resps:
    print(resp.append_result)
    if resp.append_result:
        pass

[append_result {
   offset {
     value: 3
   }
 }]